# Homework 2: Imitation Learning

Welcome to the coding portion of Homework 2! This week's problem set focuses on implementing imitation learning/behavioral cloning. The goal is that by the end of the assignment, you will know how to implement behavioral cloning and evaluate the cloned policy. As an added bonus, you will get to train an agent to play Flappy Bird!  

<center>
<img width="300px" src="https://drive.google.com/uc?id=1O4dAX_rN62fjsBRDjaYpz01ZcQ4aVP6A">
</center>

**Notes:**

* **You should be running this on Google Colab!**
* **Be sure to upload the dataset files provided with this .ipynb file using the folder tab on the left, so that you can have access to them.**


In [ ]:
# IMPORTANT: Always run this cell before anything else to ensure that you are able to access the Flappy Bird environment.
from IPython.display import clear_output

%pip install git+https://github.com/kchua/flappy-bird-gym.git
clear_output()
import flappy_bird_gym
from matplotlib import rc
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

rc('animation', html='jshtml')

def animate_images(img_lst):
  fig, ax = plt.subplots()
  ax.set_axis_off()
  ims = []
  for i, image in enumerate(img_lst):
    im = ax.imshow(image.swapaxes(0, 1), animated=True)
    if i == 0:
      ax.imshow(image.swapaxes(0, 1))
    ims.append([im])
  ani = animation.ArtistAnimation(fig, ims, interval=34)
  return ani

In [ ]:
import torch
from torch import nn
from torch.distributions import Distribution

## Problem 1: Flap Like How Flappy Sr. Taught You! (AKA Implementing Behavioral Cloning)

Growing up, Flappy Jr. has always dreamed of being as graceful as his mentor, Flappy Sr., in navigating the randomly generated pipes of the world. However, one day, his mentor simply disappeared on a daily pipe dodging adventure - eyewitness reports say that his mentor is still pipe dodging and has not reset since the day he left.

One day, as he was going about his day, he found a secret cabinet in his mentor's house containing notes of all of his pipe dodging exploits. Flappy Jr. thought to himself, "This is my chance! If I copy whatever he did in these notes, I will be as good as him."

In the following problems, we will help Flappy Jr. by implementing behavioral cloning to make use of Flappy Sr.'s notes.

### (a) Policy Evaluation

Before we get started with any training, let us make sure that we can evaluate whatever policy we get at the end. Recall that in reinforcement learning, the primary way in which we compare policies is by comparing their _returns_. The (infinite discounted) return of a policy $\pi$, denoted $R(\pi)$, is defined as

$$R(\pi) := \mathbb{E}\left[\sum_{t = 0}^\infty \gamma^tr(s_t, a_t)\right],$$

where $s_0 \sim p_0(s)$, and for all $t$, $a_t \sim \pi(s_t)$ and $s_{t + 1} \sim p(s_{t + 1} \ | \ s_t, a_t)$, and $\gamma$ is the discount factor.

**In the following code box, write a function `evaluate_policy`, which given an environment `env`, a callable `policy` which returns an action distribution given a state, and discount factor `discount`, returns a single-sample estimate of $R(\pi)$ as defined above.** `env` is assumed to follow the \*old Gym API, and the action distribution returned by `policy` is a subclass of `torch.distributions.Distribution`. You can assume that the rollout eventually terminates.

\*In newer versions of Gym/Gymnasium, `init` and `step` are assumed to return `(initial_ob, info_dict)` and `(next_ob, reward, terminated, truncated, info)`'. However, the environment we will be working with for this assignment instead returns `initial_ob` and `(next_ob, reward, done, info)`, following the old API.

In [ ]:
def evaluate_policy(env, policy, discount) -> float:
  """Returns a single-sample estimate of a policy's return.

  Args:
    env: OpenAI gym environment following old API.
    policy: Callable representing a policy.

  Returns:
    Single-sample estimate of return.
  """
  policy_return, cur_ob, done, n_steps = 0, env.reset(), False, 0
  while not done:
    action_dist = policy(cur_ob)
    action = action_dist.sample()
    cur_ob, reward, done, _ = env.step(action)
    policy_return += (discount ** n_steps) * reward
    n_steps += 1
  return policy_return

**As a test, use the function you defined in the block above to evaluate a policy which acts uniformly at random within the Flappy Bird environment over 50 independent rollouts. Use a discount of $0.999$.**

In [ ]:
env = flappy_bird_gym.make("FlappyBird-v0")

def uniform_policy(_ob):
  return torch.distributions.Categorical(logits=torch.ones(env.action_space.n))

returns = [evaluate_policy(env, uniform_policy, 0.999) for _ in range(50)]
print(np.mean(returns))

### (b) Defining a Policy

**In the code cell below, create a subclass of ``torch.nn.Module`` to represent a policy over a finite set of actions.** Remember, a policy outputs an instance of `torch.distributions.Distribution` (can be one of its subclasses), and the policy must be able to represent any distribution over the finite set of actions! **We have defined the skeleton for you below; do not modify the inputs to the functions.**

Note: Feel free to define the architecture however you like; _all we need from you is that you properly initialize the module, and that the forward method is consistent with our definition of a policy._

In [ ]:
class DiscretePolicy(nn.Module):
  def __init__(self, input_dim, n_actions):
    """Initializes a policy over the action set {0, 1, ..., n_actions-1}.

    Args:
      input_dim: Observation dimensionality.
      n_actions: Number of actions in environment.
    """
    super().__init__()
    self.linear1 = nn.Linear(input_dim, 64)
    self.linear2 = nn.Linear(64, 64)
    self.linear3 = nn.Linear(64, n_actions)

  def forward(self, ob) -> Distribution:
    """Returns a distribution over this policy's action set.
    """
    if isinstance(ob, np.ndarray):
      ob = torch.Tensor(ob)
    x = nn.functional.relu(self.linear1(ob))
    x = nn.functional.relu(self.linear2(x))
    logits = self.linear3(x)
    return torch.distributions.Categorical(logits=logits)

### (c) Setting Up Behavioral Cloning

For this problem, we will be defining a standard training loop to perform behavioral cloning. **In the following code cell, write a function which takes in:**

* **a policy `policy` of class `DiscretePolicy` to be trained,**
* **a dataset `dataset` for imitation learning,**
* **number of training steps `n_steps`,**
* **and batch size `batch_size`,**

**and returns the resulting policy after performing training according to the given parameters. Please clearly specify the format in which you expect the dataset to take within the docstring.**



In [ ]:
def train_policy_by_bc(policy, dataset, n_steps, batch_size) -> DiscretePolicy:
  """Trains the provided policy by behavioral cloning, by taking n_steps training steps with the
  provided optimizer. During training, training batches of size batch_size are sampled from the dataset
  to compute the loss.

  Args:
    policy: policy of class DiscretePolicy.
    dataset: The dataset, represented as a dictionary containing keys "obs"
      and "actions", mapping to arrays with observations and corresponding actions to
      clone, respectively. Arrays are of shape [dataset_size, ob_dim] and [dataset_size].
    n_steps: Number of training steps to take.
    batch_size: Size of the sampled batch for each training step.

  Returns:
    A policy trained according to the parameters above.
  """
  dataset_size = len(dataset["obs"])
  optimizer = torch.optim.Adam(policy.parameters(), lr=0.001)
  loss = nn.CrossEntropyLoss()
  for _ in range(n_steps):
    # Sample batch
    batch_idxs = np.random.randint(dataset_size, size=(batch_size))
    batch_ob = torch.Tensor(dataset["obs"][batch_idxs])
    batch_ac = torch.Tensor(dataset["actions"][batch_idxs]).type(torch.LongTensor)

    # Zero out optimizer gradients
    optimizer.zero_grad()

    # Get predictions
    action_dists = policy(batch_ob)

    # Compute loss and backpropagate gradients
    mean_loss = -torch.mean(action_dists.log_prob(batch_ac))
    mean_loss.backward()

    # Alternative way to compute the loss
    # mean_loss = loss(actions_dists.logits, batch_ac)

    # Adjust weights
    optimizer.step()

  return policy

### (d) Behavioral Cloning and Evaluation

The time has come for Flappy Jr. to learn from Flappy Sr.'s notes! **Make use of the provided dataset `flappy_sr_notes.mat` and obtain a policy for Flappy Jr. via behavioral cloning. Upon training, evaluate the policy using a discount of 0.999, and verify that it performs much better than selecting actions uniformly at random.** Remember to upload the provided dataset using the folder tab on the left to be able to access it!

Hint 1: You can use `scipy.io.loadmat` to load `.mat` files. Note that uploading may take some time, and `loadmat()` will error out while this process is incomplete; you can check the progress of the upload at the bottom of the folder tab to the left.  
Hint 2: Upon uploading the file, you can `right-click > Copy path` to get the path to the file within the Colab server.  
Hint 3: `flappy_sr_notes.mat` contains a record of Flappy Sr.'s whole life: every one of his rollouts is contained within the dataset in sequential order. The keys available in the dataset are `observations`, `actions`, `rewards`, `next_observations`, and `is_rollout_start` (indicates whether each index is the start of a new rollout). You may not need all keys for behavioral cloning!

In [ ]:
from scipy.io import loadmat

# Load dataset
dataset = loadmat("/content/flappy_sr_notes.mat")
dataset = {
    "obs": dataset["observations"],
    "actions": dataset["actions"][:, 0]
}

# Train policy by BC
policy = DiscretePolicy(env.observation_space.shape[0], env.action_space.n)
policy = train_policy_by_bc(policy, dataset, 20000, 64)

# Evaluate
returns = [evaluate_policy(env, policy, 0.999) for _ in range(50)]
print(np.mean(returns))

### (e) Optional (but Fun!): Visualizing Your Learned Policy Within the Game

Want to see Flappy Jr. majestically soar through the skies? We sure do! We have written for you a convenience function for visualizing a sequence of images as an animation right here within the Colab environment. **All you need to do is to copy your sampling code here from Problem 1(a) and slightly modify it to instead return a sequence of rendered images of every timestep from a rollout.** The generated animation will be saved in the folder tab to the left where you uploaded the dataset.

Hint: At any point, you can call `render(mode="rgb_array")` on a `gym` environment to obtain an RGB image of the current state.
Hint: You may need to try sampling a few rollouts!

In [ ]:
def rollout_and_render(env, policy) -> list[np.ndarray]:
  """Returns a rendering of a single rollout under the provided policy.

  Args:
    env: OpenAI gym environment following old API.
    policy: Callable representing a policy.

  Returns:
    A list of RGB images (as numpy arrays) from a single rollout.
  """
  cur_ob, done, n_steps = env.reset(), False, 0
  images = [env.render(mode="rgb_array")]
  while not done:
    action_dist = policy(cur_ob)
    action = action_dist.sample()
    cur_ob, reward, done, _ = env.step(action)
    images.append(env.render(mode="rgb_array"))
  return images

### VISUALIZE POLICY HERE!

img_lst = rollout_and_render(env, policy)

### DO NOT MODIFY ANYTHING BELOW THIS POINT
ani = animate_images(img_lst)
FFwriter = animation.FFMpegWriter(fps=30)
ani.save('animation.mp4', writer=FFwriter)

## Problem 2: Floppy the Sloppy Ruins (?) the Day (AKA An Introduction to Filtered Behavioral Cloning)

Flappy Jr. happily went to sleep, satisfied that he has had such a succesful day of flying through the skies imitating Flappy Sr. Alas, he is woken up by some strange noise in the middle of the night, and immediately realizes that someone has run off with their learned model weights. Calmly, Flappy Jr. tells himself that Flappy Sr.'s notes are just in the next room, and that they can always do behavioral cloning again. Flappy Jr. walks into the next room and is horrified to see that the notebook being scribbled over by his nemesis, Floppy the Sloppy. Upon being discovered, Floppy the Sloppy flies off awkwardly flailing through the skies, laughing while they continue their awkward motion one could hardly consider as flying.

### (a): What is Left???

Unfortunately, Floppy the Sloppy has uncharacteristically good penmanship, and his cannot be distinguished from that of Flappy Sr. As a result, we now only have a vandalized dataset `vandalized_notes.mat` consisting of the combined trajectories of Floppy the Sloppy and Flappy Sr. which are indistinguishable from each other at first glance. **For this problem, try applying behavioral cloning to train a new policy as you have done before, only this time using `vandalized-notes.mat.** How does the performance compare to your policy from Problem 1(d)?

In [ ]:
from scipy.io import loadmat

dataset = loadmat("/content/vandalized_notes.mat")
dataset = {
    "obs": dataset["observations"],
    "actions": dataset["actions"][:, 0]
}

# Train policy by BC
policy = DiscretePolicy(env.observation_space.shape[0], env.action_space.n)
policy = train_policy_by_bc(policy, dataset, 20000, 64)

returns = [evaluate_policy(env, policy, 0.999) for _ in range(50)]
print(np.mean(returns))

**Short Answer: What does this experiment tell you about running behavioral cloning on noisy/low-quality datasets? Intuitively, why does this happen?**

Hint: Think about what the BC loss is optimizing.

The quality of the cloned policy is highly dependent on the dataset, since the policy is being optimized using mean-squared error, and can thus only perform as well as the policy that generated the dataset.

### (b): Array of Hope???

While deep in thought, Flappy Jr. had a sudden epiphany; he has access to reward data! He can then weigh the importance of imitating a particular example by looking at the quality of the outcome from performing a particular action.

This strategy, where one reweighs BC examples by some predefined notion of quality, is generally referred to as **filtered behavioral cloning**. More formally, assume that we have access to a BC dataset $\{(s_1, a_1), \dots, (s_N, a_N)\}$. Furthermore, assume that we have pre-defined weights $w_1, \dots, w_N$ for each of the $N$ examples. Then, we can consider a _reweighed BC loss_

$$L_w(\pi) = \frac{1}{N}\sum_{i = 1}^{N}w_iL(\pi(s_i), a_i),$$

where $L$ is the standard per-example BC loss (e.g. cross-entropy loss for discrete actions). If the weights represent a notion of quality, one can interpret the loss above as performing filtering to ensure that only high quality examples are being imitated.

**For this problem, you will fill in the skeleton below to implement a training loop based on filtered BC. Firstly, implement the reweighed BC loss given above. Secondly, modify the BC loop you implemented before to make use of this new loss function.**

In [ ]:
class ReweighedBCLossI(nn.Module):
  def __init__(self):
    super().__init__()
    self.base_loss = nn.CrossEntropyLoss(reduction='none')   # reduction='none' is very important here!

  def forward(self, batch_predictions, batch_targets, batch_weights):
    return torch.dot(self.base_loss(batch_predictions, batch_targets), batch_weights)

# Alternative implementation that passes in the action distribution directly
class ReweighedBCLossII(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, batch_predictions, batch_targets, batch_weights):
    return -torch.dot(batch_predictions.log_prob(batch_targets), batch_weights)

In [ ]:
def train_policy_by_filtered_bc(policy, dataset, weights, n_steps, batch_size) -> DiscretePolicy:
  """Trains the provided policy by filtered behavioral cloning, by taking n_steps training steps with the
  provided optimizer with the provided weights. During training, training batches of size batch_size are sampled from the dataset
  to compute the loss.

  Args:
    policy: policy of class DiscretePolicy.
    dataset: The dataset, represented as a dictionary containing keys "obs"
      and "actions", mapping to arrays with observations and corresponding actions to
      clone, respectively. Arrays are of shape [dataset_size, ob_dim] and [dataset_size].
    weights: Weights used in the filtered BC loss.
    optimizer: An instance of torch.optim.Optimizer.
    n_steps: Number of training steps to take.
    batch_size: Size of the sampled batch for each training step.

  Returns:
    A policy trained according to the parameters above.
  """
  dataset_size = len(dataset["obs"])
  bc_loss1 = ReweighedBCLossI()   # Having both here to demonstrate how to use both implementations above
  bc_loss2 = ReweighedBCLossII()
  optimizer = torch.optim.Adam(policy.parameters(), lr=0.001)
  for _ in range(n_steps):
    # Sample dataset
    batch_idxs = np.random.randint(dataset_size, size=(batch_size,))
    batch_ob = torch.Tensor(dataset["obs"][batch_idxs])
    batch_ac = torch.Tensor(dataset["actions"][batch_idxs]).type(torch.LongTensor)
    batch_weights = torch.Tensor(weights[batch_idxs])

    # Zero out optimizer gradients
    optimizer.zero_grad()

    # Get predictions
    action_dists = policy(batch_ob)

    # Compute loss and backpropagate gradients
    mean_loss1 = bc_loss1(action_dists.logits, batch_ac, batch_weights)    # Implementation I
    mean_loss2 = bc_loss2(action_dists, batch_ac, batch_weights)           # Implementation II
    mean_loss2.backward()                                                  # Gonna use second here, but both should work.

    # Adjust weights
    optimizer.step()

  return policy

### (c) Filtering Strategy I: Trajectory-Level Reweighing???

For this problem, we will try a simple strategy: all of the $(s, a)$ pairs obtained from one trajectory will have the same weight, determined by a function of the trajectory return. To formally define this weighing scheme, let $\tau_1, \tau_2, \dots, \tau_M$ denote the trajectories found in the dataset. Then, for any $(s, a)$ contained in $\tau_i$, the corresponding weight $w$ is given by

$$w = \text{Softmax}_i\left[\frac{1}{\alpha}R(\tau_1)\ , \frac{1}{\alpha}R(\tau_2)\ , \dots\ , \frac{1}{\alpha}R(\tau_m)\right],$$

where $R(\tau_i)$ is the discounted return of trajectory $i$ and $\alpha > 0$ is a hyperparameter referred to as the _temperature_.

**Implement the function which computes the weights as defined above in the cell below, and performed filtered behavioral cloning with the computed weights.** Feel free to tune the temperature as necessary. How does the trained policy compare to that of the earlier policies in Problems 1(d) and 2(a)?

Hint: Remember that the dataset includes an array `is_rollout_start` indicating whether each datapoint is the start of a new trajectory.

In [ ]:
from scipy.special import softmax

def trajectory_level_return_weights(dataset, temp, discount) -> np.ndarray:
  """Computes an array of weights for each point in the provided dataset according to the trajectory-level reweighing scheme.

  Args:
    dataset: Input dataset.
    temp: Temperature used in softmax.
    discount: Discount used to compute return.

  Returns:
    An array of weights for each BC datapoint in the dataset.
  """
  # Useful constants
  n_trajs = np.count_nonzero(dataset["is_rollout_start"])
  n_pts = len(dataset["is_rollout_start"])

  # Compute returns for each trajectory
  traj_returns = [0 for _ in range(n_trajs)]
  traj_idx = -1
  cur_traj_len = 0
  for idx in range(n_pts):
    if dataset["is_rollout_start"][idx]:
      traj_idx += 1
      cur_traj_len = 0
    traj_returns[traj_idx] += (discount ** cur_traj_len) * dataset["rewards"][idx]
    cur_traj_len += 1

  # Compute softmax weights
  traj_weights = softmax(np.array(traj_returns) / temp)

  # Assign to each datapoint
  traj_idx = -1
  final_weights = [0 for _ in range(n_pts)]
  for idx in range(n_pts):
    if dataset["is_rollout_start"][idx]:
      traj_idx += 1
    final_weights[idx] = traj_weights[traj_idx]

  return np.array(final_weights)

In [ ]:
dataset = loadmat("/content/vandalized_notes.mat")
dataset = {
    "obs": dataset["observations"],
    "actions": dataset["actions"][:, 0],
    "rewards": dataset["rewards"][:, 0],
    "is_rollout_start": dataset["is_rollout_start"][:, 0]
}
weights = trajectory_level_return_weights(dataset, 3.0, 0.999)

# Train policy by Filtered BC
policy = DiscretePolicy(env.observation_space.shape[0], env.action_space.n)
policy = train_policy_by_filtered_bc(policy, dataset, weights, 20000, 64)

returns = [evaluate_policy(env, policy, 0.999) for _ in range(50)]
print(np.mean(returns))

**Short Answer: Why does the choice of weighing function work here? How does it affect what the behavioral cloning loss is doing?**

Since we are working here under the assumption that Floppy the Sloppy is a worse flier (and thus achieves worse return), the weighing scheme above will focus on cloning behaviors from trajectories from Flappy Sr. which have higher returns.



**Short Answer: What is the effect of the temperature on the weighing scheme?**

Hint: As a starting point, think about what happens to the softmax function as $\alpha \to 0$ (to make it even easier to think about, consider applying the softmax to two fixed values $a, b$ with $a > b$ as you do this).

As $\alpha \to 0$, the softmax function places more and more weight on cloning the high-return trajectories, becoming more and more like a hard maximum. For very large $\alpha$, trajectories are weighed equally.

**Short Answer: One could consider a version of this reweighing scheme where we remove the softmax and simply define the weight for $\tau_i$ as $R(\tau_i)$. What are the potential pitfalls of such a scheme?**

Hint: Think about the values the reward function could take in all kinds of environments.

Note that in some environments, the return might be negative if the reward function can take negative values, resulting in a nonsensical weighing scheme.

Furthermore, such a scheme is sensitive to the scale of rewards, while the softmax only takes into account the relative differences between different trajectories.

### (d) Filtering Strategy II: Truncated Future Return???

Another spark of insight just hit Flappy Jr.: rather than just defining weights at the level of trajectories, he can also weight each datapoint separately! Floppy's flying motions aren't that sloppy all the time after all, just like how a broken clock is correct twice a day. For any datapoint $(s, a)$ and its reward $r_0$, he looks at the reward $(r_1, \dots, r_{T - 1})$ of the $T - 1$ following timesteps in the same trajectory\* and computes

$$R_{T}(s, a) := \sum_{t = 0}^{T - 1}\gamma^tr_t.$$

Then, if the computed values are $R_1, \dots, R_N$ for all points in the dataset, then the weight $w_i$ of the $i^{\text{th}}$ point is given by

$$w_i = N\cdot \text{Softmax}_i\left[\frac{1}{\alpha}R_1\ , \frac{1}{\alpha}R_2\ , \dots\ , \frac{1}{\alpha}R_N\right],$$

where $\alpha$ is the temperature hyperparameter (as in the previous part). Note that there are now two hyperparameters, the truncation horizon $T$, and the temperature $\alpha$.

\* **If there are fewer than $T - 1$ timesteps after $(s, a)$ in the trajectory that $(s,a)$ belongs to, the remaining timestep rewards in the sum are set to $0$.**

**Implement the function which computes the weights as defined above in the cell below, and performed filtered behavioral cloning with the computed weights.** Feel free to tune the temperature as necessary. How does the trained policy compare to that of the earlier policies in Problems 1(d) and 2(a)?

In [ ]:
from scipy.special import softmax

def truncated_future_return_weights(dataset, truncation_horizon, temp, discount) -> np.ndarray:
  """Computes an array of weights for each point in the provided dataset according to the truncated future return reweighing scheme.

  Args:
    dataset: Input dataset.
    truncation_horizon: How many timesteps to consider for computing future return.
    temp: Temperature used in softmax.
    discount: Discount used to compute return.

  Returns:
    An array of weights for each BC datapoint in the dataset.
  """
  n_pts = len(dataset["is_rollout_start"])

  # Compute truncated returns from each datapoint
  # Note: Not the most efficient implementation - can use numpy's sliding window function with some
  #   clever transformation of a sliding window view of is_rollout_start array to vectorize this.
  all_returns = [0 for _ in range(n_pts)]
  for idx in range(n_pts):
    for offset in range(truncation_horizon):
      if offset > 0 and (idx + offset >= n_pts or dataset["is_rollout_start"][idx + offset]):
        break
      all_returns[idx] += (discount ** offset) * dataset["rewards"][idx + offset]

  softmax_weights = softmax(np.array(all_returns) / temp)
  return n_pts * softmax_weights

In [ ]:
from scipy.io import loadmat

dataset = loadmat("/content/vandalized_notes.mat")
dataset = {
    "obs": dataset["observations"],
    "actions": dataset["actions"][:, 0],
    "rewards": dataset["rewards"][:, 0],
    "is_rollout_start": dataset["is_rollout_start"][:, 0]
}
weights = truncated_future_return_weights(dataset, 100, 3.0, 0.999)

# Train policy by Filtered BC
policy = DiscretePolicy(env.observation_space.shape[0], env.action_space.n)
policy = train_policy_by_filtered_bc(policy, dataset, weights, 20000, 64)

returns = [evaluate_policy(env, policy, 0.999) for _ in range(50)]
print(np.mean(returns))

**Short Answer: Let us explore the effect of the hyperparameter $T$ on the weighing scheme. Give a succinct description of the weights when $T = 1$. Do you expect this to work well in general? Why or why not?**

When $T = 1$, each timestep is weighed by the corresponding reward at that timestep. This would not work well in general since this would result in "myopic" policies that only focus on immediate rewards rather than long-term returns.

**Short Answer: Can you think of a reason why one can set $T$ relatively small in Flappy Bird and still obtain decent performance?**

Hint: Think about the structure of the problem the agent is trying to solve.

Small $T$ can work well for this environment due to the periodic nature of the game: as long as an action is helpful for clearing the next pipe, whatever happens in the future after that is not as important for determining whether an action is "useful".

### (e) Optional: Other Reweighing Schemes???

Feel free to try implementing other reweighing schemes here! Options to try:

1. Only training on the top $q\%$ of trajectories ranked by return.
2. Using the future return within the trajectory from the current timestep.

In [ ]:
def my_reweighing_scheme(dataset, *args, **kwargs) -> np.ndarray:
  """Computes an array of weights for each point in the provided dataset according to the truncated future return reweighing scheme.

  Args:
    dataset: Input dataset.
    *args: Replace with your scheme's hyperparameters.
    **kwargs: Replace with your scheme's hyperparameters.

  Returns:
    An array of weights for each BC datapoint in the dataset.
  """
  ### YOUR CODE HERE!
  pass


### YOUR CODE HERE: RUN FILTERED BC BELOW!